In [73]:
import numpy as np
import re


data = """
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3"""

nrows, ncols = 7, 11
qrow, qcol = int((nrows - 1)/2), int((ncols - 1)/2)

lines = data.strip().split("\n")
nums = [list(map(int, re.findall(r'-?\d+', s))) for s in lines]

prs = np.array([n[1] for n in nums])
pcs = np.array([n[0] for n in nums])
vrs = np.array([n[3] for n in nums])
vcs = np.array([n[2] for n in nums])

tiles = np.zeros((nrows, ncols))

prs, pcs

(array([4, 3, 3, 0, 0, 0, 6, 0, 3, 3, 4, 5]),
 array([ 0,  6, 10,  2,  0,  3,  7,  3,  9,  7,  2,  9]))

In [74]:
#for i, p, v in zip(range(len(positions)), positions, velocities):

def wrap(a, b, max_value):
    return np.mod((a + b), max_value)

for _ in range(100):
    prs = wrap(prs, vrs, nrows)
    pcs = wrap(pcs, vcs, ncols)

list(zip(prs, pcs))

[(5, 3),
 (4, 5),
 (0, 9),
 (5, 4),
 (6, 1),
 (3, 1),
 (0, 6),
 (3, 2),
 (2, 0),
 (0, 6),
 (5, 4),
 (6, 6)]

In [75]:
for pr, pc in zip(prs, pcs):
    tiles[pr, pc] += 1

tiles

array([[0., 0., 0., 0., 0., 0., 2., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 2., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [82]:
np.product(
    (
        tiles[:qrow, :qcol].sum(),
        tiles[qrow + 1:, :qcol].sum(),
        tiles[:qrow, qcol + 1:].sum(),
        tiles[qrow + 1:, qcol + 1:].sum()
    )
)

12.0